In [211]:
import pandas as pd
import os

In [212]:
#paths

data= r"C:\Users\admin\python\Horse Race Prediction"

In [213]:
#loading forward data

forward = pd.read_csv("forward.csv")

In [214]:
#load and combine horses and race dataset

horse_race_data=[]

for year in range(1990,2021):
    horse_file=os.path.join(data,f"horses_{year}.csv")
    race_file=os.path.join(data,f"races_{year}.csv")

    if os.path.exists(horse_file) and os.path.exists(race_file):
        horses =pd.read_csv(horse_file)
        races =pd.read_csv(race_file)

#          #merge horses and races datasets on 'rid'
#         combined_data = pd.merge(horses, races, on="rid", how="inner")
#         horse_race_data.append(combined_data)

# #combine into a single DataFrame
# df = pd.concat(horse_race_data, ignore_index=True)

 #save dataset into csv
# horses.to_csv("horses.csv",index=False)
# races.to_csv("races.csv",index=False)
# print("saved successfully")

C:\Users\admin\AppData\Local\Temp\ipykernel_15900\2932159393.py:10: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  horses =pd.read_csv(horse_file)
C:\Users\admin\AppData\Local\Temp\ipykernel_15900\2932159393.py:10: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  horses =pd.read_csv(horse_file)


In [215]:
# #load combined data
# file_name= "horse_race.csv"
# df=pd.read_csv(file_name)

In [216]:
# data cleaning

In [217]:

# Checking for missing values in races DataFrame

print("Missing values in Races DataFrame:")
print(races.isnull().mean()*100)

# Checking for missing values in horses DataFrame

print("\nMissing values in Horses DataFrame:")
print(horses.isnull().mean()*100)

# Checking for missing values in forward DataFrame

print("\nMissing values in Forward DataFrame:")
print(forward.isnull().mean()*100)
     

Missing values in Races DataFrame:
rid             0.000000
course          0.000000
time            0.000000
date            0.000000
title           0.000000
rclass         51.358659
band           63.951602
ages            0.000000
distance        0.000000
condition       0.000000
hurdles        79.268622
prizes          0.000000
winningTime     0.000000
prize           0.000000
metric          0.000000
countryCode     0.000000
ncond           0.000000
class           0.000000
currency       37.265107
dtype: float64

Missing values in Horses DataFrame:
rid              0.000000
horseName        0.000000
age              0.000000
saddle           0.004013
decimalPrice     0.000000
isFav            0.000000
trainerName      0.014046
jockeyName       0.002007
position         0.000000
positionL       14.197428
dist            24.046070
weightSt         0.000000
weightLb         0.000000
overWeight      96.722693
outHandicap     99.094393
headGear        62.690201
RPR             48.454

In [218]:
#dropping columns which are 50% above null values

threshold =len(horses)*0.50
horses=horses.dropna(thresh=threshold,axis=1)

threshold =len(races)*0.50
races=races.dropna(thresh=threshold,axis=1)

threshold =len(forward)*0.50
forward=forward.dropna(thresh=threshold,axis=1)

In [219]:
horses.isnull().mean()*100

rid              0.000000
horseName        0.000000
age              0.000000
saddle           0.004013
decimalPrice     0.000000
isFav            0.000000
trainerName      0.014046
jockeyName       0.002007
position         0.000000
positionL       14.197428
dist            24.046070
weightSt         0.000000
weightLb         0.000000
RPR             48.454649
OR              46.796600
father           0.000000
mother           0.000669
gfather          0.024078
runners          0.000000
margin           0.000000
weight           0.000000
res_win          0.000000
res_place        0.000000
price           36.453017
dtype: float64

In [220]:
horses["positionL"].value_counts()

positionL
.5       12031
nk       11160
.75       9946
1.25      7977
1         7261
         ...  
91           2
2.0          1
7.0          1
107          1
24.75        1
Name: count, Length: 171, dtype: int64

In [221]:
#mapping for non-numeric terms in positionL

mapping={
  "shd":0.1,
   "hd":0.2,
   "nk":0.3
}

#replacing
horses["positionL"] =horses["positionL"].replace(mapping)
#convert the column to numeric
horses["positionL"]=pd.to_numeric(horses["positionL"],errors="coerce")

In [222]:
horses["positionL"].isnull().sum()

25640

In [223]:
horses['positionL'].describe()

count    123873.000000
mean          3.813783
std           7.431375
min           0.100000
25%           0.500000
50%           1.500000
75%           3.750000
max         107.000000
Name: positionL, dtype: float64

In [224]:
horses['positionL'].fillna(horses['positionL'].median(),inplace=True)

C:\Users\admin\AppData\Local\Temp\ipykernel_15900\3644971839.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  horses['positionL'].fillna(horses['positionL'].median(),inplace=True)


In [225]:
horses["dist"].describe()

count     113561
unique       819
top         4.25
freq        2083
Name: dist, dtype: object

In [226]:
#coverting string into numeric
horses["dist"] =pd.to_numeric(horses["dist"],errors="coerce")


In [227]:
horses["dist"].describe()

count    113397.000000
mean         14.912465
std          18.235040
min           0.250000
25%           4.500000
50%           8.750000
75%          17.500000
max         241.250000
Name: dist, dtype: float64

In [228]:
#coverting nan into median
horses["dist"].fillna(horses["dist"].median(),inplace=True)

C:\Users\admin\AppData\Local\Temp\ipykernel_15900\1492920984.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  horses["dist"].fillna(horses["dist"].median(),inplace=True)


In [280]:
horses["RPR"].describe()

count    77067.000000
mean        74.677073
std         29.092444
min          1.000000
25%         55.000000
50%         73.000000
75%         95.000000
max        177.000000
Name: RPR, dtype: float64

In [284]:
horses["RPR"].fillna(horses["RPR"].median(),inplace=True)

C:\Users\admin\AppData\Local\Temp\ipykernel_15900\1589162635.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  horses["RPR"].fillna(horses["RPR"].median(),inplace=True)


In [286]:
horses["OR"].describe()

count    79546.000000
mean        80.446685
std         25.672796
min          1.000000
25%         61.000000
50%         77.000000
75%         97.000000
max        177.000000
Name: OR, dtype: float64

In [288]:
horses["OR"].fillna(horses["OR"].median(),inplace=True)

C:\Users\admin\AppData\Local\Temp\ipykernel_15900\514480544.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  horses["OR"].fillna(horses["OR"].median(),inplace=True)


In [229]:
horses["price"].value_counts()

price
12/1      3392
14/1      3347
16/1      3174
10/1      2869
33/1      2802
          ... 
123/20       1
254/1        1
236/1        1
240/1        1
296/1        1
Name: count, Length: 578, dtype: int64

In [290]:
#dropping remaining features where missing rate is less than 1%
horses.dropna(axis=1,inplace=True)

In [292]:
horses.isnull().mean()*100

rid             0.0
horseName       0.0
age             0.0
decimalPrice    0.0
isFav           0.0
position        0.0
positionL       0.0
dist            0.0
weightSt        0.0
weightLb        0.0
RPR             0.0
OR              0.0
father          0.0
runners         0.0
margin          0.0
weight          0.0
res_win         0.0
res_place       0.0
dtype: float64

In [232]:
horses["decimalPrice"].describe()

count    149513.000000
mean          0.117114
std           0.117515
min           0.001427
25%           0.034483
50%           0.076923
75%           0.161290
max           0.965517
Name: decimalPrice, dtype: float64

In [233]:
horses["decimalPrice"]=horses["decimalPrice"].round(4)

In [234]:
horses["isFav"].value_counts()

isFav
0    133950
1     15563
Name: count, dtype: int64

In [235]:
win_rate_favorites = (horses[horses["isFav"] == 1]["position"] == 1).mean()
win_rate_non_favorites = (horses[horses["isFav"] == 0]["position"] == 1).mean()

print(f"Favorites' Win Rate: {win_rate_favorites:.2%}")
print(f"Non-Favorites' Win Rate: {win_rate_non_favorites:.2%}")


Favorites' Win Rate: 32.26%
Non-Favorites' Win Rate: 7.25%


In [236]:
horses["positionL"].min() 

0.1

In [237]:
horses["positionL"].max() 

107.0

In [238]:
mapping = {
    "nse": 0.05,   # Nose
    "nk": 0.25,    # Neck
    "hd": 0.10,    # Head
    "shd": 0.08,   # Short head
    "dist": 20,    # Distance (assume a large margin)
    "snk": 0.20,   # Short neck
    "dnt": None    # Did not try (set to NaN)
}
horses["positionL"] = horses["positionL"].replace(mapping)


In [239]:
#ensuring all values are numeric
horses["positionL"] = pd.to_numeric(horses["positionL"], errors="coerce")

In [240]:
horses["dist"].describe()  

count    149513.000000
mean         13.423875
std          16.098191
min           0.250000
25%           5.750000
50%           8.750000
75%          13.750000
max         241.250000
Name: dist, dtype: float64

In [241]:
horses["weightSt"].describe()  

count    149513.000000
mean          9.105569
std           1.048855
min           7.000000
25%           8.000000
50%           9.000000
75%          10.000000
max          16.000000
Name: weightSt, dtype: float64

In [242]:
races.isnull().mean()*100

rid             0.000000
course          0.000000
time            0.000000
date            0.000000
title           0.000000
ages            0.000000
distance        0.000000
condition       0.000000
prizes          0.000000
winningTime     0.000000
prize           0.000000
metric          0.000000
countryCode     0.000000
ncond           0.000000
class           0.000000
currency       37.265107
dtype: float64

In [243]:
races["currency"].value_counts()

currency
GBP    7460
EUR    1821
Name: count, dtype: int64